In [1]:
!nvidia-smi

Sun Apr 13 18:42:27 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-12GB           On  | 00000000:03:00.0 Off |                    0 |
| N/A   34C    P0              25W / 250W |      0MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU'

import numpy as np

import time
import torch
from ultralytics import YOLO

In [3]:
num_gpus = torch.cuda.device_count()
print(f"Number of GPUs available: {num_gpus}")
if num_gpus < 4:
    print("Warning: Fewer than 4 GPUs are available. Adjust 'device' parameter accordingly.")


Number of GPUs available: 4


In [4]:
model = YOLO("yolov8n.yaml")
data_config_path = "/home/temgar.s/pneumonia_yolo.yaml"
print(f"Using data configuration from: {data_config_path}")

Using data configuration from: /home/temgar.s/pneumonia_yolo.yaml


In [5]:
epochs = 10       
imgsz = 256       
batch_size = 16   
device = "0,1,2,3" 

print(f"Starting training with parameters:")
print(f"  Epochs: {epochs}")
print(f"  Image Size: {imgsz}")
print(f"  Batch Size: {batch_size}")
print(f"  Device(s): {device}")

Starting training with parameters:
  Epochs: 10
  Image Size: 256
  Batch Size: 16
  Device(s): 0,1,2,3


In [6]:
start_time = time.time()

try:
    results = model.train(
        data=data_config_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch_size,
        device=device
    )
except Exception as e:
    print("An error occurred during training:")
    print(e)
    raise

end_time = time.time()
total_training_time = end_time - start_time
print(f"\nTotal Training Time: {total_training_time:.2f} seconds")

Ultralytics 8.3.107 🚀 Python-3.12.4 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-12GB, 12194MiB)
                                                       CUDA:1 (Tesla P100-PCIE-12GB, 12194MiB)
                                                       CUDA:2 (Tesla P100-PCIE-12GB, 12194MiB)
                                                       CUDA:3 (Tesla P100-PCIE-12GB, 12194MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/home/temgar.s/pneumonia_yolo.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=256, save=True, save_period=-1, cache=False, device=0,1,2,3, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dn

train: Scanning /home/temgar.s/yolo_dataset/labels/train.cache... 26680 images, 20670 backgrounds, 0 corrupt: 100%|██████████| 26680/26680 [00:00<?, ?it/s]
/home/temgar.s/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
val: Scanning /home/temgar.s/yolo_dataset/labels/train.cache... 26680 images, 20670 backgrounds, 0 corrupt: 100%|██████████| 26680/26680 [00:00<?, ?it/s]
/home/temgar.s/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is

Plotting labels to runs/detect/train6/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 256 train, 256 val
Using 32 dataloader workers
Logging results to runs/detect/train6
Starting training for 10 epochs...
Closing dataloader mosaic


/home/temgar.s/.local/lib/python3.12/site-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all      26680       9541      0.427      0.254      0.268      0.105

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10     0.422G      1.355      1.796      1.078          0        256: 100%|██████████| 1668/1668 [04:44<00:00,  5.86it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3335/3335 [02:58<00:00, 18.67it/s]


                   all      26680       9541      0.379      0.273      0.261      0.106

10 epochs completed in 1.307 hours.
Optimizer stripped from runs/detect/train6/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train6/weights/best.pt, 6.2MB

Validating runs/detect/train6/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.12.4 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-12GB, 12194MiB)
                                                       CUDA:1 (Tesla P100-PCIE-12GB, 12194MiB)
                                                       CUDA:2 (Tesla P100-PCIE-12GB, 12194MiB)
                                                       CUDA:3 (Tesla P100-PCIE-12GB, 12194MiB)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3335/3335 [02:46<00:00, 20.03it/s]


                   all      26680       9541      0.427      0.254      0.268      0.105
Speed: 0.1ms preprocess, 2.2ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/train6

Total Training Time: 5084.88 seconds


In [7]:
baseline_time = 500.0  # example value in seconds
speedup = baseline_time / total_training_time
efficiency = speedup / 4  # assuming 4 GPUs
print(f"Speedup: {speedup:.2f}")
print(f"Parallel Efficiency: {efficiency:.2f}")

Speedup: 0.10
Parallel Efficiency: 0.02
